### PART 2

#### Interesting to test and compare 3 to 4 different non linear models (maybe testing more but only 3 to 4 in the report), try to not have 3 models of the same 'family' 

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance 
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def compute_rmse(y_pred, y_true):
    return root_mean_squared_error(y_true, y_pred)

def optimize_model(model, params, X_train, y_train, scoring_metric='neg_root_mean_squared_error'):
    """
    Optimise les hyperparamètres du modèle en utilisant GridSearchCV.
    """
    # GridSearchCV utilise la validation croisée (CV) pour trouver la meilleure combinaison de paramètres
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=params,
        scoring=scoring_metric,
        cv=5, # 5-fold cross-validation
        n_jobs=-1,
        verbose=1
    )
    
    # Entraînement et optimisation
    grid_search.fit(X_train, y_train)
    
    # Affichage des meilleurs résultats
    best_score = -grid_search.best_score_ # Le score est négatif car nous maximisons un score d'erreur
    print(f"Meilleurs paramètres trouvés: {grid_search.best_params_}")
    print(f"RMSE de Cross-Validation (Moyenne): {best_score:.4f}")
    
    return grid_search.best_estimator_, best_score



In [ ]:
print("\n--- Optimisation du Random Forest ---")
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None], # None = expande jusqu'à ce que les feuilles soient pures
    'min_samples_split': [2, 5]
}

best_rf_model, cv_rmse_rf = optimize_model(RandomForestRegressor(random_state=42), 
                                          rf_params, 
                                          X_train_full_scaled_df, 
                                          y_train_full)
rf_test_pred = best_rf_model.predict(X_test_scaled_df)
rmse_test_rf = compute_rmse(rf_test_pred, y_test)
print(f"Random Forest RMSE sur le jeu de TEST: {rmse_test_rf:.4f}")

In [ ]:
print("\n--- Optimisation du K-NN ---")
knn_params = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'], # Poids uniformes ou pondérés par l'inverse de la distance
    'p': [1, 2] # 1 pour distance de Manhattan, 2 pour distance Euclidienne
}

best_knn_model, cv_rmse_knn = optimize_model(KNeighborsRegressor(), 
                                            knn_params, 
                                            X_train_full_scaled_df, 
                                            y_train_full)
knn_test_pred = best_knn_model.predict(X_test_scaled_df)
rmse_test_knn = compute_rmse(knn_test_pred, y_test)
print(f"K-NN RMSE sur le jeu de TEST: {rmse_test_knn:.4f}")

In [ ]:
print("\n--- Optimisation du SVR ---")
svr_params = {
    'C': [0.1, 1, 10], # Paramètre de régularisation
    'gamma': ['scale', 'auto'], # Coefficient du noyau (kernel)
    'kernel': ['rbf']
}

best_svr_model, cv_rmse_svr = optimize_model(SVR(), 
                                           svr_params, 
                                           X_train_full_scaled_df, 
                                           y_train_full)
svr_test_pred = best_svr_model.predict(X_test_scaled_df)
rmse_test_svr = compute_rmse(svr_test_pred, y_test)
print(f"SVR RMSE sur le jeu de TEST: {rmse_test_svr:.4f}")

In [ ]:
print("\n--- Optimisation du MLP (Simple) ---")
mlp_params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 25)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001], # Terme de régularisation L2
    'max_iter': [300]
}

# Le MLP peut nécessiter une mise à l'échelle (déjà fait ici) et est très sensible aux initialisations.
best_mlp_model, cv_rmse_mlp = optimize_model(MLPRegressor(random_state=42), 
                                           mlp_params, 
                                           X_train_full_scaled_df, 
                                           y_train_full)
mlp_test_pred = best_mlp_model.predict(X_test_scaled_df)
rmse_test_mlp = compute_rmse(mlp_test_pred, y_test)
print(f"MLP RMSE sur le jeu de TEST: {rmse_test_mlp:.4f}")

In [ ]:
# Rassembler les résultats des meilleurs modèles
results = {
    "Random Forest": {'model': best_rf_model, 'rmse': rmse_test_rf},
    "K-NN": {'model': best_knn_model, 'rmse': rmse_test_knn},
    "SVR": {'model': best_svr_model, 'rmse': rmse_test_svr},
    "MLP": {'model': best_mlp_model, 'rmse': rmse_test_mlp},
}

# Trouver le meilleur modèle
best_model_name = min(results, key=lambda k: results[k]['rmse'])
best_model = results[best_model_name]['model']

print(f"\n=======================================================")
print(f"Le meilleur modèle non linéaire global est: {best_model_name} avec RMSE = {results[best_model_name]['rmse']:.4f}")
print(f"=======================================================")

# --- ANALYSE D'IMPORTANCE PAR PERMUTATION ---
# Appliquée au meilleur modèle global
print(f"\n--- Importance par Permutation pour le meilleur modèle: {best_model_name} ---")

result = permutation_importance(
    best_model, 
    X_test_scaled_df, 
    y_test, 
    n_repeats=30, # Augmenter les répétitions pour plus de stabilité
    random_state=42, 
    n_jobs=-1
)

# Créer un DataFrame pour la visualisation
sorted_idx = result.importances_mean.argsort()[::-1]
feature_importance_df = pd.DataFrame({
    'Feature': X_test_scaled_df.columns[sorted_idx],
    'Importance_Mean': result.importances_mean[sorted_idx],
    'Importance_Std': result.importances_std[sorted_idx]
})

print(feature_importance_df)

#